In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, BatchNormalization, ReLU, Dropout
from tensorflow.keras.models import Model
from itertools import product

import functions as fs
from functions import make_state, run, get_optimal_value, get_optimal_action, get_optimal_actions, get_model_actions, test_model, one_batch, test_model_accuracy, one_batch_supervised, train, train_supervised, create_model

# 单步奖励收集

这是一个用于练习策略梯度法 (policy gradient method) 的玩具题。  
8个格子首尾相连，其中随机3个格子奖励，奖励数值为0~1的均匀随机数。  
动作：选择一个格子。
计分规则：  
1. 如果格子有奖励（大于0），获得对应的奖励。
2. 如果格子是空的（等于0），且两侧的格子都有奖励，会获得两侧格子的总奖励。
3. 否则奖励为0。

## 测试hyperparameters

In [2]:
for optimilizer in ['sgd', 'adam']:
    for n_hidden_layers in [1, 2]:
        for n_dense_units in [64, 128, 256, 512]:
            for ratio_dropout in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
                model = create_model(n_hidden_layers, n_dense_units, ratio_dropout, optimilizer)
                scores = np.array([one_batch(model, batch_size=128, n_test_rounds=10000) for i in np.arange(300)])
                argmax = scores.argmax()
                print(optimilizer, n_hidden_layers, n_dense_units, ratio_dropout, argmax, scores[argmax])

sgd 1 64 0.2 204 0.8248358598306655
sgd 1 64 0.3 159 0.8305548017766521
sgd 1 64 0.4 231 0.8280183309475164
sgd 1 64 0.5 292 0.8177246414354393
sgd 1 64 0.6 217 0.8160378401679262
sgd 1 64 0.7 207 0.8307612579383953
sgd 1 128 0.2 181 0.8500731975526788
sgd 1 128 0.3 196 0.8590271956423483
sgd 1 128 0.4 164 0.8403888276563491
sgd 1 128 0.5 197 0.8415861312996321
sgd 1 128 0.6 175 0.849156561100324
sgd 1 128 0.7 241 0.8570490655679734
sgd 1 256 0.2 137 0.8767178277734814
sgd 1 256 0.3 163 0.8751898127784395
sgd 1 256 0.4 196 0.8929618962541235
sgd 1 256 0.5 183 0.8678954191929861
sgd 1 256 0.6 155 0.8776852263183684
sgd 1 256 0.7 190 0.855041421452595
sgd 1 512 0.2 177 0.9002695710875036
sgd 1 512 0.3 143 0.8980001387826704
sgd 1 512 0.4 168 0.8888414414734253
sgd 1 512 0.5 169 0.8973560204168048
sgd 1 512 0.6 173 0.8997983320175176
sgd 1 512 0.7 214 0.878271239572213
sgd 2 64 0.2 290 0.8030141960436504
sgd 2 64 0.3 201 0.8129056165957603
sgd 2 64 0.4 178 0.8031456692799241
sgd 2 64 0.5 